# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [251]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [252]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [253]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [254]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',) 
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [255]:
# Read the XML
document = ET.parse( './data/mondial_database.xml' )

##  Objective 1: Find 10 countries with lowest infant morality rates.
*****
**Approach:** Loop through the document nodes to create a dictionary with `country` as the key and `infant_mortality` as the value, then convert this dict to a DataFrame. Use the DataFrame to get the 10 countries with the lowest infant mortality rates

 
 

In [256]:
#initialize a dictionary
countries_dict = {}
countries_df = pd.DataFrame(columns=('country','infant_mortality'))

#loop through all the country nodes in the document
for element in document.iterfind('country'):
    ctry = element.find('name')
    int_mort = element.find('infant_mortality')
    #discard if the country node is not present
    if (ctry) != None:
        #discard if there is no infant_mortality is present
        if (int_mort) != None:
            countries_dict[ctry.text]=int_mort.text
        else:
            countries_dict[ctry.text]=np.NaN

#create a DataFrame from the dictionary            
countries_df=pd.DataFrame.from_dict(countries_dict, orient='index')

#set the column name
countries_df.columns=['InfantMortality']

In [257]:
#convert the values to numbers
countries_df.InfantMortality = pd.to_numeric(countries_df.InfantMortality,errors='raise')

In [258]:
#get the 10 countries with the lowest infant mortality rates
countries_df.sort_values(by = 'InfantMortality',ascending=True).head(10)

,InfantMortality
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


## Objective 2 : 10 cities with the largest population.
****
**Approach:** The objective is to get the latest population of all the cities and for each city, get the latest population (i.e. population of  the most recent year). The structure of the XML to is 

`Countries --> Provinces -->City-->Population(for multiple years)`

The `province` node may or may not be present, if it is not present, the `city`  will be directly under countries. The high level steps to get the 10 cities with the largest population are
1. Setup a loop to go through each `country`
2. Create a dictionary of all provinces [key - id ; value - province name]
3. Go through each `city` in the `country`
4. Get the latest `population` entry
5. Collect all these values (`country, city, province, year, population`) in a list; use the dictionary created in step 2
6. At the end of the loop create a DataFrame from the list
7. Use the DataFrame to get the cities with the largest population


In [259]:
#create a list to store the country, city, year and population, this list will be convereted to a DataFrame when data 
#for all cities are captured
cit_pop_list = []

#start a loop to go through all the countries in the XML
for element in document.iterfind('country'):
    ctry = element.find('name').text
    '''
    Countries may or may not have provinces, and there are some countries where the city name is not unique. e.g.
    Richmond in United States. So the only way to uniquely identify a city to show the city along with the country
    and the province.
    
    we store the provice names of a country in a dictionary here. This dictionary will be used to fill in the 
    province names. Storing the provinces here reduces the amount of code required.   
    '''
  
    prov_dict = {}
    for province in element.findall('province'):
        prov_dict[province.attrib['id']] = province.find('name').text
        
    # setup a loop for all the cities within a country    
    for subelement in element.getiterator('city'):
        city = subelement.find('name')
        if city != None: # check to see if 'city' element is present
            # for each city there may be more than one population entry, the below code stores only the latest
            # population entry, i.e. entry whose year value is max
            yr = 0
            pop = 0
            #get the population elements for each city
            for popelement in subelement.getiterator('population'):
                if popelement != None: # check to see if any 'population' element is present
                    #store the population for the year that is the greatest for this particular city
                    if int(popelement.attrib['year']) > yr:
                        yr = int(popelement.attrib['year'])
                        pop = int(popelement.text)
                        #if this city has provinces, store the province name from the dict values created
                        #earlier in the program
                        if 'province' in subelement.attrib:
                            prov = prov_dict[subelement.attrib['province']]
                        else:
                            prov = 'No Province'
            #create a list with the elements and append to the list
            cit_pop_list.append([ctry,city.text,prov,int(popelement.attrib['year']),int(popelement.text)])
        
#once the list is constructed, create a DataFrame from the list
cit_pop_df=pd.DataFrame.from_records(cit_pop_list)

#set the column names
cit_pop_df.columns=['Country','City','Province','Year','Population']


cit_pop_df.head(10)

,Country,City,Province,Year,Population
0,Albania,Tirana,No Province,2011,418495
1,Albania,Shkodër,No Province,2011,77075
2,Albania,Durrës,No Province,2011,113249
3,Albania,Vlorë,No Province,2011,79513
4,Albania,Elbasan,No Province,2011,78703
5,Albania,Korçë,No Province,2011,51152
6,Greece,Komotini,No Province,2011,51152
7,Greece,Kavala,Anatolikis Makedonias kai Thrakis,2011,58790
8,Greece,Athina,Attikis,2011,664046
9,Greece,Peiraias,Attikis,2011,163688


In [260]:
cit_pop_df.sort_values(by = 'Population',ascending=False).head(10)

,Country,City,Province,Year,Population
1341,China,Shanghai,Shanghai,2010,22315474
771,Turkey,Istanbul,İstanbul,2012,13710512
1527,India,Mumbai,Maharashtra,2011,12442373
479,Russia,Moskva,Moscow,2013,11979529
1340,China,Beijing,Beijing,2010,11716620
2810,Brazil,São Paulo,São Paulo,2010,11152344
1342,China,Tianjin,Tianjin,2010,11090314
1064,China,Guangzhou,Guangdong,2010,11071424
1582,India,Delhi,Delhi,2011,11034555
1067,China,Shenzhen,Guangdong,2010,10358381



## Objective 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
****
**Approach:** Setup a list to collect the values, loop through the XML to collect the values in the list, then finally create a DataFrame from the list to get the top 10 ethnic groups. High level steps
1. Loop through each `country` 
2. For each `country` store the latest population and the year
3. Loop through each `ethnicgroup` in the `country`
4. For each `ethnicgroup` get the population by using the `percentage` and population stored in step 2

In [273]:
ethgrp_list = []

for element in document.iterfind('country'):
    ctry = element.find('name').text
    yr=0
    #find the latest population
    for popele in element.findall('population'):
        if int(popele.attrib['year']) > yr:
            popltn = int(popele.text)
            yr = int(popele.attrib['year']) 
    
    #store the ethnic group and its population     
    for ethgrp in element.findall('ethnicgroup'):
        percent = float(ethgrp.attrib['percentage']) / 100
        ethgrp_list.append([ctry,ethgrp.text, int(yr), (float(popltn)*percent)])
        
        
#once the list is constructed, create a DataFrame from the list
ethgrp_df=pd.DataFrame.from_records(ethgrp_list)

#set the column names
ethgrp_df.columns=['Country','EthnicGroup','Year','Population']

ethgrp_df.head(5)

,Country,EthnicGroup,Year,Population
0,Albania,Albanian,2011,2.660131e+06
1,Albania,Greek,2011,8.400414e+04
2,Greece,Greek,2011,1.005915e+07
3,Macedonia,Macedonian,2011,1.322388e+06
4,Macedonia,Albanian,2011,5.190681e+05


In [275]:
#use groupby on ethnicgroup and population to find the largest ethnic groups
ethgrp_df[['EthnicGroup','Population']].groupby(['EthnicGroup']).sum().sort_values(by='Population',ascending=False).head(10)

,Population
EthnicGroup,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


## Objective 4. Name and country of a) longest river, b) largest lake and c) airport at highest elevation
****
**Approach:** A similar approach will be used for all 3 elements. First step will be to create a dictionary of country ids and country names, this dictionary will be used when updating the record with the country name. Once the country dictionary is created, loop through each of the elements (river, lake, airport), gather the element values, add to the list. Finally use the list to create a DataFrame and use the DataFrame to get the target information

In [263]:
'''
create a country dict to capture the country codes and the country names
this dict will be used later to update the country names for rivers, lakes and airports
those nodes contain only the country codes
'''

ctry_dict={}
for ctry_element in document.iterfind('country'):
    ctry_dict[ctry_element.attrib['car_code']]=ctry_element.find('name').text
    

#create a DataFrame of rivers and their countries-------------------------------------
  
#create a list to store the values - river name, country code, country name, river length
river_country_list = []

#setup a loop to go through the river nodes
for riv_element in document.iterfind('river'):
    river = riv_element.find('name').text
    river_length_el = riv_element.find('length')
    #discard if the river does not have the length element
    if river_length_el != None:
        river_length = river_length_el.text
        #some rivers run through multiple countries, each country code is seperated by a space
        #split the country codes by string and add a record for each country
        for riv_country_code in riv_element.attrib['country'].split():
            river_country = ctry_dict[riv_country_code]
            river_country_list.append([river,riv_country_code,river_country,float(river_length)]) 
        

#once the list is constructed, create a DataFrame from the list
river_country_df=pd.DataFrame.from_records(river_country_list)

#set the column names
river_country_df.columns=['River','CountryCode','CountryName','Length']





#country and their lakes--------------------------------------------------------------

#create a list to hold the values  - lake, country code, country name, lake area
lake_country_list=[] 

#loop through the 'lake' nodes
for lake_element in document.iterfind('lake') :
    lake_name = (lake_element.find('name').text)
    lake_area_el = lake_element.find('area')
    #some some of the area elements are not present, ignore those nodes
    if lake_area_el != None:
        #some lakes are across multiple countries, country codes are seperated by a space
        #add a record for each country code
        for lake_country_code in lake_element.attrib['country'].split():
            lake_country = ctry_dict[lake_country_code]
            #build the list - add the values
            lake_country_list.append([lake_name,lake_country_code,lake_country,float(lake_area_el.text)])

#create a DataFrame from the list
lake_country_df = pd.DataFrame.from_records(lake_country_list)

#set the column names
lake_country_df.columns=['Lake','CountryCode','CountryName','Area']



#country and their airports--------------------------------------------------------------
#create a list to hold the values - airport, country code, country and airport elevation
airport_country_list=[]

#loop through all the airport nodes
for airport_element in document.iterfind('airport'):
    airport_name = airport_element.find('name').text
    airport_country_code = airport_element.attrib['country']
    airport_country = ctry_dict[airport_country_code]
    airport_elevation_el = airport_element.find('elevation')
    #discard all elements where the elevation element is absent
    if airport_elevation_el != None:
        airport_elevation = airport_elevation_el.text
        #some elevation dont have text
        if ((airport_elevation is  not  None)):
            airport_country_list.append([airport_name, airport_country_code, airport_country, float(airport_elevation)])

#create a DataFrame from the list
airport_country_df = pd.DataFrame.from_records(airport_country_list)
airport_country_df.columns = ['Airport','CountryCode','CountryName','Elevation']



In [264]:
lake_country_df.head()

,Lake,CountryCode,CountryName,Area
0,Inari,SF,Finland,1040.0
1,Oulujaervi,SF,Finland,928.0
2,Kallavesi,SF,Finland,472.0
3,Saimaa,SF,Finland,4370.0
4,Paeijaenne,SF,Finland,1118.0


In [265]:
river_country_df.head()

,River,CountryCode,CountryName,Length
0,Thjorsa,IS,Iceland,230.0
1,Joekulsa a Fjoellum,IS,Iceland,206.0
2,Glomma,N,Norway,604.0
3,Lagen,N,Norway,322.0
4,Goetaaelv,S,Sweden,93.0


In [266]:
airport_country_df.head()

,Airport,CountryCode,CountryName,Elevation
0,Herat,AFG,Afghanistan,977.0
1,Kabul Intl,AFG,Afghanistan,1792.0
2,Tirana Rinas,AL,Albania,38.0
3,Cheikh Larbi Tebessi,DZ,Algeria,811.0
4,Batna Airport,DZ,Algeria,822.0


In [267]:
#Country with the longest river
river_country_df.iloc[river_country_df['Length'].idxmax()]

River          Amazonas
CountryCode          CO
CountryName    Colombia
Length             6448
Name: 298, dtype: object

In [268]:
river_country_df[river_country_df.River=='Amazonas']

,River,CountryCode,CountryName,Length
298,Amazonas,CO,Colombia,6448.0
299,Amazonas,BR,Brazil,6448.0
300,Amazonas,PE,Peru,6448.0


In [269]:
#country with largest lake
lake_country_df.iloc[lake_country_df['Area'].idxmax()]

Lake           Caspian Sea
CountryCode              R
CountryName         Russia
Area                386400
Name: 68, dtype: object

In [270]:
lake_country_df[lake_country_df.Lake=='Caspian Sea']

,Lake,CountryCode,CountryName,Area
68,Caspian Sea,R,Russia,386400.0
69,Caspian Sea,AZ,Azerbaijan,386400.0
70,Caspian Sea,KAZ,Kazakhstan,386400.0
71,Caspian Sea,IR,Iran,386400.0
72,Caspian Sea,TM,Turkmenistan,386400.0


In [271]:
#country with highest elevation airport
airport_country_df.iloc[airport_country_df.Elevation.idxmax()]

Airport        El Alto Intl
CountryCode             BOL
CountryName         Bolivia
Elevation              4063
Name: 80, dtype: object

In [272]:
airport_country_df[airport_country_df.Airport=='El Alto Intl']

,Airport,CountryCode,CountryName,Elevation
80,El Alto Intl,BOL,Bolivia,4063.0
